In [2]:
"""
FINAL FBref Selenium Scraper - Barcelona games only, includes Started status
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random

def get_chrome_driver(headless=True):
    """Create a new Chrome driver instance"""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
    
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)


class CompleteFBrefScraper:
    def __init__(self):
        self.players = {
            'Lewandowski': 'https://fbref.com/en/players/8d78e732/matchlogs/2024-2025/summary/Robert-Lewandowski-Match-Logs',
            'Yamal': 'https://fbref.com/en/players/82ec26c1/matchlogs/2024-2025/Lamine-Yamal-Match-Logs',
            'Raphinha': 'https://fbref.com/en/players/3423f250/matchlogs/2024-2025/Raphinha-Match-Logs',
            'Pedri': 'https://fbref.com/en/players/0d9b2d31/matchlogs/2024-2025/Pedri-Match-Logs'
        }
    
    def scrape_player(self, player_name, url):
        """Scrape one player's match logs with fresh browser"""
        print(f"\n📊 Scraping {player_name}...")
        print(f"   🚀 Starting new browser for {player_name}...")
        
        driver = None
        try:
            driver = get_chrome_driver(headless=True)
            driver.get(url)
            
            wait_time = random.uniform(10, 15)
            print(f"   ⏳ Waiting {wait_time:.1f} seconds...")
            time.sleep(wait_time)
            
            table = driver.find_element(By.CSS_SELECTOR, "table.stats_table")
            rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")
            
            matches = []
            for row in rows:
                row_class = row.get_attribute('class') or ''
                if 'spacer' in row_class or not row.text.strip():
                    continue
                
                try:
                    cells = row.find_elements(By.TAG_NAME, "td")
                    if len(cells) < 5:
                        continue
                    
                    match_data = {
                        'Player': player_name,
                        'Date': row.find_element(By.TAG_NAME, "th").text.strip(),
                    }
                    
                    for cell in cells:
                        stat_name = cell.get_attribute('data-stat')
                        value = cell.text.strip()
                        if stat_name and value:
                            match_data[stat_name] = value
                    
                    if match_data.get('Date') and len(match_data) > 3:
                        matches.append(match_data)
                    
                except Exception as e:
                    continue
            
            df = pd.DataFrame(matches)
            
            # Remove empty rows
            df = df.dropna(how='all')
            cols_to_check = [col for col in df.columns if col not in ['Player', 'Date']]
            df = df.dropna(subset=cols_to_check, how='all')
            
            # FILTER: Keep only Barcelona games (team column with value 'Barcelona')
            before_filter = len(df)
            df = df[df['team'] == 'Barcelona'].copy()
            after_filter = len(df)
            print(f"   🔍 Filtered: {before_filter} total → {after_filter} Barcelona games only")
            
            print(f"   ✅ Got {len(df)} Barcelona matches for {player_name}")
            
            return df
            
        except Exception as e:
            print(f"   ❌ Error scraping {player_name}: {str(e)}")
            return None
            
        finally:
            if driver:
                driver.quit()
                print(f"   🔒 Closed browser for {player_name}")
    
    def scrape_all(self):
        """Scrape all 4 players with fresh browser each"""
        all_data = []
        
        for player_name, url in self.players.items():
            df = self.scrape_player(player_name, url)
            if df is not None and len(df) > 0:
                all_data.append(df)
            
            wait_time = random.uniform(10, 15)
            print(f"\n   ⏳ Waiting {wait_time:.1f} seconds before next player...\n")
            time.sleep(wait_time)
        
        if all_data:
            combined = pd.concat(all_data, ignore_index=True)
            print(f"\n✅ Total: {len(combined)} Barcelona matches collected from {len(all_data)} players")
            return combined
        return None
    
    def save_data(self, df, filename='fbref_barcelona_only.csv'):
        """Save and clean the data"""
        if df is None or len(df) == 0:
            print("No data to save")
            return
    
        # Complete column mapping
        rename_map = {
            'date': 'Date',
            'day_of_week': 'Day',
            'comp': 'Competition',
            'round': 'Round',
            'venue': 'Venue',
            'result': 'Result',
            'team': 'Team',
            'opponent': 'Opponent',
            'game_started': 'Started',
            'position': 'Position',
            'minutes': 'Min',
            'goals': 'Gls',
            'assists': 'Ast',
            'pens_made': 'PK',
            'pens_att': 'PKatt',
            'shots': 'Sh',
            'shots_on_target': 'SoT',
            'cards_yellow': 'CrdY',
            'cards_red': 'CrdR',
            'touches': 'Touches',
            'tackles': 'Tkl',
            'interceptions': 'Int',
            'blocks': 'Blocks',
            'xg': 'xG',
            'npxg': 'npxG',
            'xg_assist': 'xAG',
            'sca': 'SCA',
            'gca': 'GCA',
            'passes_completed': 'Cmp',
            'passes': 'Att',
            'passes_pct': 'Cmp%',
            'progressive_passes': 'PrgP',
            'carries': 'Carries',
            'progressive_carries': 'PrgC',
            'take_ons': 'Take-Ons_Att',
            'take_ons_won': 'Take-Ons_Succ'
        }
    
        df = df.rename(columns=rename_map)
        
        # All important columns in order (PLAYER FIRST!)
        desired_cols = [
            'Player', 'Date', 'Day', 'Competition', 'Round', 'Venue', 'Result', 
            'Team', 'Opponent', 'Started', 'Position', 'Min',
            'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT',
            'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
            'xG', 'npxG', 'xAG', 'SCA', 'GCA',
            'Cmp', 'Att', 'Cmp%', 'PrgP',
            'Carries', 'PrgC', 'Take-Ons_Att', 'Take-Ons_Succ'
        ]
        
        # Keep only columns that exist
        available = [col for col in desired_cols if col in df.columns]
        df_clean = df[available].copy()
        
        # Remove any remaining empty rows
        df_clean = df_clean.dropna(how='all')
        df_clean = df_clean[df_clean['Date'].notna()]
        
        # Final filter: Keep only Barcelona games
        if 'Team' in df_clean.columns:
            df_clean = df_clean[df_clean['Team'] == 'Barcelona'].copy()
        
        # Convert numeric columns
        numeric_cols = [col for col in df_clean.columns if col not in 
                    ['Date', 'Day', 'Competition', 'Round', 'Venue', 'Result', 
                        'Team', 'Opponent', 'Started', 'Position', 'Player']]
        for col in numeric_cols:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        
        # Save
        df_clean.to_csv(filename, index=False)
        print(f"\n💾 Saved to: {filename}")
        print(f"   Rows: {len(df_clean)}")
        print(f"   Columns: {len(df_clean.columns)}")
        print(f"\n📋 Columns included:")
        for col in df_clean.columns:
            print(f"   ✓ {col}")
        
        # Show sample
        print(f"\n📊 Sample data:")
        display_cols = [c for c in ['Player', 'Date', 'Opponent', 'Result', 'Started', 'Min', 'Gls', 'Ast'] if c in df_clean.columns]
        print(df_clean[display_cols].head(5))
        
        # Show player breakdown
        if 'Player' in df_clean.columns:
            print(f"\n📈 Barcelona matches per player:")
            print(df_clean['Player'].value_counts())
        
        # Show starter breakdown
        if 'Started' in df_clean.columns and 'Player' in df_clean.columns:
            print(f"\n🏃 Games started per player:")
            started_counts = df_clean.groupby('Player')['Started'].apply(lambda x: (x == 'Y').sum())
            print(started_counts)
        
        return df_clean

def main():
    print("\n" + "="*60)
    print("FC BARCELONA COMPLETE DATA SCRAPER")
    print("Barcelona games only + Started status")
    print("="*60)
    
    try:
        scraper = CompleteFBrefScraper()
        data = scraper.scrape_all()
        
        if data is not None:
            scraper.save_data(data)
            print("\n🎉 SUCCESS!")
            print("\n✅ All Barcelona games scraped (international games removed)")
            print("✅ Started column included")
            print("✅ All 31+ metrics collected")
            print("\nNext: Collect SofaScore ratings separately")
        else:
            print("\n❌ No data collected")
            
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")


if __name__ == "__main__":
    main()


FC BARCELONA COMPLETE DATA SCRAPER
Barcelona games only + Started status

📊 Scraping Lewandowski...
   🚀 Starting new browser for Lewandowski...
   ⏳ Waiting 11.8 seconds...
   🔍 Filtered: 58 total → 39 Barcelona games only
   ✅ Got 39 Barcelona matches for Lewandowski
   🔒 Closed browser for Lewandowski

   ⏳ Waiting 10.8 seconds before next player...


📊 Scraping Yamal...
   🚀 Starting new browser for Yamal...
   ⏳ Waiting 12.2 seconds...
   🔍 Filtered: 65 total → 42 Barcelona games only
   ✅ Got 42 Barcelona matches for Yamal
   🔒 Closed browser for Yamal

   ⏳ Waiting 12.9 seconds before next player...


📊 Scraping Raphinha...
   🚀 Starting new browser for Raphinha...
   ⏳ Waiting 13.7 seconds...
   🔍 Filtered: 66 total → 43 Barcelona games only
   ✅ Got 43 Barcelona matches for Raphinha
   🔒 Closed browser for Raphinha

   ⏳ Waiting 12.1 seconds before next player...


📊 Scraping Pedri...
   🚀 Starting new browser for Pedri...
   ⏳ Waiting 11.7 seconds...
   🔍 Filtered: 70 total 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import json
import time
import random

class SofaScoreJSONScraper:
    def __init__(self):
        self.players = {
            'Pedri': {
                'id': '992587',
                'tournaments': {
                    'La Liga': {'tournament_id': '8', 'season_id': '61643'},
                    'Champions League': {'tournament_id': '7', 'season_id': '61644'},
                    'Copa del Rey': {'tournament_id': '329', 'season_id': '66885'},
                    'Supercopa': {'tournament_id': '213', 'season_id': '66001'}
                }
            },
            'Lewandowski': {
                'id': '41789',
                'tournaments': {
                    'La Liga': {'tournament_id': '8', 'season_id': '61643'},
                    'Champions League': {'tournament_id': '7', 'season_id': '61644'},
                    'Copa del Rey': {'tournament_id': '329', 'season_id': '66885'},
                    'Supercopa': {'tournament_id': '213', 'season_id': '66001'}
                }
            },
            'Yamal': {
                'id': '1402912',
                'tournaments': {
                    'La Liga': {'tournament_id': '8', 'season_id': '61643'},
                    'Champions League': {'tournament_id': '7', 'season_id': '61644'},
                    'Copa del Rey': {'tournament_id': '329', 'season_id': '66885'},
                    'Supercopa': {'tournament_id': '213', 'season_id': '66001'}
                }
            },
            'Raphinha': {
                'id': '831005',
                'tournaments': {
                    'La Liga': {'tournament_id': '8', 'season_id': '61643'},
                    'Champions League': {'tournament_id': '7', 'season_id': '61644'},
                    'Copa del Rey': {'tournament_id': '329', 'season_id': '66885'},
                    'Supercopa': {'tournament_id': '213', 'season_id': '66001'}
                }
            }
        }

        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--log-level=3")
        self.driver = webdriver.Chrome(options=chrome_options)

    def get_player_ratings(self, player_name, player_id, tournament_id, season_id, tournament_name):
        url = f"https://www.sofascore.com/api/v1/player/{player_id}/unique-tournament/{tournament_id}/season/{season_id}/ratings/overall"
        self.driver.get(url)
        time.sleep(2)  # wait for JSON to load

        print(f"   📊 {tournament_name}...", end=' ')
        try:
            json_text = self.driver.find_element("tag name", "pre").text
            data = json.loads(json_text)

            matches = []
            for match in data.get("seasonRatings", []):
                event = match.get("event", {})
                rating = match.get("rating")
                date = pd.to_datetime(match.get("startTimestamp"), unit='s').strftime('%Y-%m-%d')
                opponent = match.get("opponent", {}).get("name", "")
                matches.append({
                    "Player": player_name,
                    "Competition": tournament_name,
                    "Date": date,
                    "Opponent": opponent,
                    "Rating": rating
                })

            print(f"✅ {len(matches)} matches")
            return matches

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            return []

    def scrape_player(self, player_name, player_info):
        all_matches = []
        print(f"\n📊 Scraping {player_name}...")
        player_id = player_info['id']

        for tournament_name, t in player_info['tournaments'].items():
            matches = self.get_player_ratings(player_name, player_id, t['tournament_id'], t['season_id'], tournament_name)
            all_matches.extend(matches)
            time.sleep(random.uniform(1,2))

        if all_matches:
            return pd.DataFrame(all_matches)
        return None

    def scrape_all(self):
        all_data = []
        print("\n==============================\nSOFASCORE RATINGS SCRAPER (JSON API)\n==============================")
        for player_name, player_info in self.players.items():
            df = self.scrape_player(player_name, player_info)
            if df is not None:
                all_data.append(df)
            time.sleep(random.uniform(2,4))

        self.driver.quit()
        if all_data:
            combined = pd.concat(all_data, ignore_index=True)
            print(f"\n✅ Total ratings collected: {len(combined)}")
            return combined
        return None

    def save_data(self, df, filename='sofascore_ratings.csv'):
        if df is None or len(df)==0:
            print("No data to save")
            return
        df = df.drop_duplicates(subset=['Player','Date','Opponent'])
        df.to_csv(filename, index=False)
        print(f"\n💾 Saved to {filename}")
        print(df.head(10))
        return df


if __name__ == "__main__":
    scraper = SofaScoreJSONScraper()
    data = scraper.scrape_all()
    if data is not None:
        scraper.save_data(data)
        print("\n🎉 SUCCESS! All player ratings collected")
    else:
        print("\n❌ No data collected")



SOFASCORE RATINGS SCRAPER (JSON API)

📊 Scraping Pedri...
   📊 La Liga... ✅ 37 matches
   📊 Champions League... ✅ 14 matches
   📊 Copa del Rey... ✅ 6 matches
   📊 Supercopa... ✅ 2 matches

📊 Scraping Lewandowski...
   📊 La Liga... ✅ 34 matches
   📊 Champions League... ✅ 13 matches
   📊 Copa del Rey... ✅ 3 matches
   📊 Supercopa... ✅ 2 matches

📊 Scraping Yamal...
   📊 La Liga... ✅ 35 matches
   📊 Champions League... ✅ 13 matches
   📊 Copa del Rey... ✅ 5 matches
   📊 Supercopa... ✅ 2 matches

📊 Scraping Raphinha...
   📊 La Liga... ✅ 36 matches
   📊 Champions League... ✅ 14 matches
   📊 Copa del Rey... ✅ 5 matches
   📊 Supercopa... ✅ 2 matches

✅ Total ratings collected: 223

💾 Saved to sofascore_ratings.csv
  Player Competition        Date         Opponent  Rating
0  Pedri     La Liga  2025-05-25    Athletic Club     7.2
1  Pedri     La Liga  2025-05-18       Villarreal     7.6
2  Pedri     La Liga  2025-05-15         Espanyol     7.7
3  Pedri     La Liga  2025-05-11      Real Madrid  

In [3]:
"""
FINAL FBref Selenium Scraper - Barcelona games only, includes Started status
Fixed: No matches are dropped, empty stats allowed, all tables scraped
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random

def get_chrome_driver(headless=True):
    """Create a new Chrome driver instance"""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
    
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)


class CompleteFBrefScraper:
    def __init__(self):
        self.players = {
            'Lewandowski': 'https://fbref.com/en/players/8d78e732/matchlogs/2024-2025/summary/Robert-Lewandowski-Match-Logs',
            'Yamal': 'https://fbref.com/en/players/82ec26c1/matchlogs/2024-2025/Lamine-Yamal-Match-Logs',
            'Raphinha': 'https://fbref.com/en/players/3423f250/matchlogs/2024-2025/Raphinha-Match-Logs',
            'Pedri': 'https://fbref.com/en/players/0d9b2d31/matchlogs/2024-2025/Pedri-Match-Logs'
        }
    
    def scrape_player(self, player_name, url):
        """Scrape one player's match logs with fresh browser"""
        print(f"\n📊 Scraping {player_name}...")
        
        driver = None
        try:
            driver = get_chrome_driver(headless=True)
            driver.get(url)
            
            wait_time = random.uniform(12, 18)  # wait for all tables
            print(f"   ⏳ Waiting {wait_time:.1f} seconds for tables to load...")
            time.sleep(wait_time)
            
            tables = driver.find_elements(By.CSS_SELECTOR, "table.stats_table")
            matches = []

            for table in tables:
                rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")
                for row in rows:
                    try:
                        # Only skip rows with no date
                        date = row.find_element(By.TAG_NAME, "th").text.strip()
                        if not date:
                            continue
                    except:
                        continue

                    match_data = {'Player': player_name, 'Date': date}
                    cells = row.find_elements(By.TAG_NAME, "td")
                    for cell in cells:
                        stat_name = cell.get_attribute('data-stat')
                        value = cell.text.strip()
                        if stat_name:
                            match_data[stat_name] = value

                    # Only include rows with 'team' or 'opponent'
                    if 'team' in match_data and match_data['team']:
                        matches.append(match_data)
            
            df = pd.DataFrame(matches)

            # Filter Barcelona games (case-insensitive)
            df = df[df['team'].str.contains('Barcelona', case=False, na=False)].copy()

            print(f"   ✅ Got {len(df)} Barcelona matches for {player_name}")
            return df
            
        except Exception as e:
            print(f"   ❌ Error scraping {player_name}: {str(e)}")
            return None
            
        finally:
            if driver:
                driver.quit()
    
    def scrape_all(self):
        """Scrape all 4 players with fresh browser each"""
        all_data = []
        
        for player_name, url in self.players.items():
            df = self.scrape_player(player_name, url)
            if df is not None and len(df) > 0:
                all_data.append(df)
            
            wait_time = random.uniform(10, 15)
            print(f"\n   ⏳ Waiting {wait_time:.1f} seconds before next player...\n")
            time.sleep(wait_time)
        
        if all_data:
            combined = pd.concat(all_data, ignore_index=True)
            print(f"\n✅ Total: {len(combined)} Barcelona matches collected from {len(all_data)} players")
            return combined
        return None
    
    def save_data(self, df, filename='fbref_barcelona_only.csv'):
        """Save and clean the data"""
        if df is None or len(df) == 0:
            print("No data to save")
            return

        # Rename and order columns (partial example)
        rename_map = { 'date':'Date', 'comp':'Competition', 'team':'Team', 'opponent':'Opponent', 'game_started':'Started', 'minutes':'Min', 'goals':'Gls', 'assists':'Ast' }
        df = df.rename(columns=rename_map)

        # Keep only Barcelona games
        if 'Team' in df.columns:
            df = df[df['Team'].str.contains('Barcelona', case=False, na=False)]

        df.to_csv(filename, index=False)
        print(f"\n💾 Saved to: {filename}")
        print(f"   Rows: {len(df)}, Columns: {len(df.columns)}")
        print("\n📋 Sample data:")
        print(df.head(5))
        return df

def main():
    print("\n" + "="*60)
    print("FC BARCELONA COMPLETE DATA SCRAPER")
    print("="*60)
    
    scraper = CompleteFBrefScraper()
    data = scraper.scrape_all()
    
    if data is not None:
        scraper.save_data(data)
        print("\n🎉 SUCCESS! All Barcelona matches scraped")
    else:
        print("\n❌ No data collected")

if __name__ == "__main__":
    main()



FC BARCELONA COMPLETE DATA SCRAPER

📊 Scraping Lewandowski...
   ⏳ Waiting 12.3 seconds for tables to load...
   ✅ Got 52 Barcelona matches for Lewandowski

   ⏳ Waiting 12.5 seconds before next player...


📊 Scraping Yamal...
   ⏳ Waiting 14.2 seconds for tables to load...
   ✅ Got 55 Barcelona matches for Yamal

   ⏳ Waiting 11.2 seconds before next player...


📊 Scraping Raphinha...
   ⏳ Waiting 15.7 seconds for tables to load...
   ✅ Got 57 Barcelona matches for Raphinha

   ⏳ Waiting 13.5 seconds before next player...


📊 Scraping Pedri...
   ⏳ Waiting 14.1 seconds for tables to load...
   ✅ Got 59 Barcelona matches for Pedri

   ⏳ Waiting 14.2 seconds before next player...


✅ Total: 223 Barcelona matches collected from 4 players

💾 Saved to: fbref_barcelona_only.csv
   Rows: 223, Columns: 38

📋 Sample data:
        Player        Date dayofweek Competition        round venue result  \
0  Lewandowski  2024-08-17       Sat     La Liga  Matchweek 1  Away  W 2–1   
1  Lewandowski  2

In [7]:
import pandas as pd

# Load datasets
fbref_df = pd.read_csv("fbref_barcelona_only.csv")
sofascore_df = pd.read_csv("sofascore_ratings.csv")

# Standardize dates
fbref_df['Date'] = pd.to_datetime(fbref_df['Date'])
sofascore_df['Date'] = pd.to_datetime(sofascore_df['Date'])

# Merge on Player + Date only
merged_df = pd.merge(
    fbref_df,
    sofascore_df[['Player', 'Date', 'Rating']],  # only keep relevant columns
    how='left',
    on=['Player', 'Date']
)

# Sort by player and date
merged_df = merged_df.sort_values(['Player', 'Date']).reset_index(drop=True)

# Save
merged_df.to_csv("fbref_sofascore_merged_by_date_only.csv", index=False)

print(f"Merged dataset saved. Total rows: {len(merged_df)}")
print(merged_df.head(10))


Merged dataset saved. Total rows: 223
        Player       Date dayofweek   Competition         round venue result  \
0  Lewandowski 2024-08-17       Sat       La Liga   Matchweek 1  Away  W 2–1   
1  Lewandowski 2024-08-24       Sat       La Liga   Matchweek 2  Home  W 2–1   
2  Lewandowski 2024-08-27       Tue       La Liga   Matchweek 3  Away  W 2–1   
3  Lewandowski 2024-08-31       Sat       La Liga   Matchweek 4  Home  W 7–0   
4  Lewandowski 2024-09-15       Sun       La Liga   Matchweek 5  Away  W 4–1   
5  Lewandowski 2024-09-19       Thu  Champions Lg  League phase  Away  L 1–2   
6  Lewandowski 2024-09-22       Sun       La Liga   Matchweek 6  Away  W 5–1   
7  Lewandowski 2024-09-25       Wed       La Liga   Matchweek 7  Home  W 1–0   
8  Lewandowski 2024-09-28       Sat       La Liga   Matchweek 8  Away  L 2–4   
9  Lewandowski 2024-10-01       Tue  Champions Lg  League phase  Home  W 5–0   

           Team        Opponent Started  ... passes_completed  passes  \
0     Ba

In [10]:
print(merged_df.shape)
print(merged_df.isna().sum())

print(merged_df.dtypes)

print(merged_df.describe())



(223, 39)
Player                  0
Date                    0
dayofweek               0
Competition             0
round                   0
venue                   0
result                  0
Team                    0
Opponent                0
Started                 0
position                2
Min                     0
Gls                     0
Ast                     0
pens_made               0
pens_att                0
shots                   0
shots_on_target         0
cards_yellow            0
cards_red               0
touches                27
tackles                27
interceptions           0
blocks                 27
xg                     27
npxg                   27
xg_assist              27
sca                    27
gca                    27
passes_completed       27
passes                 27
passes_pct             27
progressive_passes     27
carries                27
progressive_carries    27
take_ons               27
take_ons_won           27
match_report            0
Ra

In [ ]:
"""
FC Barcelona Player Analysis - Enhanced Exploratory Data Analysis
Added: KDE, Violin Plots, Spearman Correlation, Heatmaps, Transformations, Outlier Detection
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)


class BarcelonaEDA:
    def __init__(self, data_file):
        """Load data"""
        self.df = pd.read_csv(data_file)
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        
        # Define metric categories
        self.attacking_metrics = ['Gls', 'Ast', 'shots', 'shots_on_target', 'xg', 'npxg', 'xg_assist']
        self.creative_metrics = ['sca', 'gca', 'progressive_passes', 'progressive_carries', 
                                'passes_completed', 'passes_pct', 'take_ons', 'take_ons_won']
        self.defensive_metrics = ['tackles', 'interceptions', 'blocks']
        self.discipline_metrics = ['cards_yellow', 'cards_red']
        
        print(f"Loaded {len(self.df)} matches for {self.df['Player'].nunique()} players")
        print(f"Date range: {self.df['Date'].min()} to {self.df['Date'].max()}")
    
    def data_overview(self):
        """Basic data overview"""
        print("\n" + "="*60)
        print("DATA OVERVIEW")
        print("="*60)
        
        print(f"\nShape: {self.df.shape}")
        print(f"\nColumns: {list(self.df.columns)}")
        
        print("\nMissing Values:")
        missing = self.df.isnull().sum()
        print(missing[missing > 0])
        
        print("\nData Types:")
        print(self.df.dtypes)
        
        print("\nBasic Statistics:")
        print(self.df[['Rating', 'Min', 'Gls', 'Ast', 'xg', 'sca']].describe())
    
    def player_summary(self):
        """Summary statistics by player"""
        print("\n" + "="*60)
        print("PLAYER SUMMARIES")
        print("="*60)
        
        for player in self.df['Player'].unique():
            player_data = self.df[self.df['Player'] == player]
            
            print(f"\n{player}:")
            print(f"  Matches: {len(player_data)}")
            print(f"  Avg Rating: {player_data['Rating'].mean():.2f} (±{player_data['Rating'].std():.2f})")
            print(f"  Games Started: {(player_data['Started'] == 'Y').sum()}")
            print(f"  Total Goals: {player_data['Gls'].sum()}")
            print(f"  Total Assists: {player_data['Ast'].sum()}")
            print(f"  Avg Minutes: {player_data['Min'].mean():.1f}")
    
    def match_result_analysis(self):
        """Analyze performance by match result"""
        print("\n" + "="*60)
        print("PERFORMANCE BY MATCH RESULT")
        print("="*60)
        
        # Count results
        print("\nMatch Results:")
        results = self.df['result'].value_counts()
        print(results)
        
        # Overall average rating by result
        print("\n" + "-"*60)
        print("OVERALL: Average Rating by Result")
        print("-"*60)
        for result_type in ['W', 'D', 'L']:
            mask = self.df['result'].str.contains(result_type, na=False)
            if mask.sum() > 0:
                avg_rating = self.df.loc[mask, 'Rating'].mean()
                std_rating = self.df.loc[mask, 'Rating'].std()
                count = mask.sum()
                print(f"  {result_type}: {avg_rating:.2f} (±{std_rating:.2f}) - {count} matches")
        
        # Individual player ratings by result
        print("\n" + "-"*60)
        print("INDIVIDUAL PLAYER: Average Rating by Result")
        print("-"*60)
        
        for player in self.df['Player'].unique():
            print(f"\n{player}:")
            player_data = self.df[self.df['Player'] == player]
            
            for result_type in ['W', 'D', 'L']:
                mask = player_data['result'].str.contains(result_type, na=False)
                if mask.sum() > 0:
                    avg_rating = player_data.loc[mask, 'Rating'].mean()
                    std_rating = player_data.loc[mask, 'Rating'].std()
                    count = mask.sum()
                    print(f"  {result_type}: {avg_rating:.2f} (±{std_rating:.2f}) - {count} matches")
    
    def correlation_analysis(self):
        """Correlation between metrics and ratings - Both Pearson and Spearman"""
        print("\n" + "="*60)
        print("CORRELATION ANALYSIS")
        print("="*60)
        
        # Overall correlations
        all_metrics = (self.attacking_metrics + self.creative_metrics + self.discipline_metrics)
        
        print("\nOVERALL Pearson Correlation with Rating:")
        print("-" * 60)
        
        pearson_correlations = []
        for metric in all_metrics:
            if metric in self.df.columns:
                mask = self.df[metric].notna() & self.df['Rating'].notna()
                if mask.sum() > 10:
                    corr_p, _ = stats.pearsonr(
                        self.df.loc[mask, metric], 
                        self.df.loc[mask, 'Rating']
                    )
                    corr_s, _ = spearmanr(
                        self.df.loc[mask, metric],
                        self.df.loc[mask, 'Rating']
                    )
                    
                    pearson_correlations.append({
                        'Metric': metric,
                        'Pearson': corr_p,
                        'Spearman': corr_s
                    })
                    
                    print(f"{metric:25s}: Pearson r = {corr_p:6.3f}, Spearman ρ = {corr_s:6.3f}")
        
        # Individual player correlations by category
        print("\n" + "-"*60)
        print("INDIVIDUAL PLAYER: Attacking vs Creative Impact")
        print("-"*60)
        
        for player in self.df['Player'].unique():
            print(f"\n{player}:")
            player_data = self.df[self.df['Player'] == player]
            
            # Attacking correlation
            attacking_corrs = []
            for metric in self.attacking_metrics:
                if metric in player_data.columns:
                    mask = player_data[metric].notna() & player_data['Rating'].notna()
                    if mask.sum() > 5:
                        corr, _ = stats.pearsonr(
                            player_data.loc[mask, metric],
                            player_data.loc[mask, 'Rating']
                        )
                        attacking_corrs.append(abs(corr))
            
            # Creative correlation
            creative_corrs = []
            for metric in self.creative_metrics:
                if metric in player_data.columns:
                    mask = player_data[metric].notna() & player_data['Rating'].notna()
                    if mask.sum() > 5:
                        corr, _ = stats.pearsonr(
                            player_data.loc[mask, metric],
                            player_data.loc[mask, 'Rating']
                        )
                        creative_corrs.append(abs(corr))
            
            avg_attacking = np.mean(attacking_corrs) if attacking_corrs else 0
            avg_creative = np.mean(creative_corrs) if creative_corrs else 0
            
            print(f"  Avg Attacking Correlation: {avg_attacking:.3f}")
            print(f"  Avg Creative Correlation:  {avg_creative:.3f}")
            
            if avg_attacking > avg_creative * 1.1:
                print(f"  ➜ ATTACKING impact dominates")
            elif avg_creative > avg_attacking * 1.1:
                print(f"  ➜ CREATIVE impact dominates")
            else:
                print(f"  ➜ BALANCED impact")
        
        return pd.DataFrame(pearson_correlations).sort_values('Pearson', ascending=False)
    
    def outlier_detection(self):
        """Detect outliers using IQR method"""
        print("\n" + "="*60)
        print("OUTLIER DETECTION (IQR Method)")
        print("="*60)
        
        metrics_to_check = ['Rating', 'Gls', 'Ast', 'xg', 'sca']
        
        for metric in metrics_to_check:
            if metric in self.df.columns:
                print(f"\n{metric}:")
                data = self.df[metric].dropna()
                
                Q1 = data.quantile(0.25)
                Q3 = data.quantile(0.75)
                IQR = Q3 - Q1
                
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                outliers = self.df[(self.df[metric] < lower_bound) | (self.df[metric] > upper_bound)]
                
                print(f"  Q1 = {Q1:.2f}, Q3 = {Q3:.2f}, IQR = {IQR:.2f}")
                print(f"  Lower Bound = {lower_bound:.2f}, Upper Bound = {upper_bound:.2f}")
                print(f"  Number of outliers: {len(outliers)}")
                
                if len(outliers) > 0:
                    print(f"  Outlier matches:")
                    for _, row in outliers[['Player', 'Date', 'Opponent', metric]].head(5).iterrows():
                        print(f"    {row['Player']}: {row['Date'].strftime('%Y-%m-%d')} vs {row['Opponent']} - {metric}={row[metric]}")
    
    def visualize_ratings(self):
        """Rating distribution visualizations"""
        print("\n" + "="*60)
        print("CREATING VISUALIZATIONS")
        print("="*60)
        
        # 1. Rating distributions with KDE
        self.plot_rating_distributions_kde()
        
        # 2. Correlation Heatmap
        self.plot_correlation_heatmap()
        
        # 3. Violin plots by result
        self.plot_violin_by_result()
        
        # 4. Attacking metrics scatter plots (PER PLAYER)
        self.plot_scatter_attacking_per_player()
        
        # 5. Creative metrics scatter plots (PER PLAYER)
        self.plot_scatter_creative_per_player()
        
        # 6. Discipline analysis
        self.plot_discipline_analysis()
        
        # 7. Player comparison
        self.plot_player_comparison()
        
        # 8. Individual ratings by result
        self.plot_individual_ratings_by_result()
        
        # 9. Attacking vs Creative impact comparison
        self.plot_attacking_vs_creative_impact()
    
    def plot_rating_distributions_kde(self):
        """Rating distributions with Kernel Density Estimation"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Player Rating Distributions with KDE', fontsize=16, fontweight='bold')
        
        players = self.df['Player'].unique()
        for idx, player in enumerate(players):
            ax = axes[idx // 2, idx % 2]
            player_data = self.df[self.df['Player'] == player]
            
            # Histogram
            ax.hist(player_data['Rating'], bins=15, edgecolor='black', alpha=0.5, 
                   color='skyblue', density=True, label='Histogram')
            
            # KDE
            from scipy.stats import gaussian_kde
            kde = gaussian_kde(player_data['Rating'].dropna())
            x_range = np.linspace(player_data['Rating'].min(), player_data['Rating'].max(), 100)
            ax.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
            
            # Mean line
            ax.axvline(player_data['Rating'].mean(), color='darkred', linestyle='--', linewidth=2,
                      label=f'Mean: {player_data["Rating"].mean():.2f}')
            
            ax.set_title(f'{player} - {len(player_data)} matches', fontsize=12, fontweight='bold')
            ax.set_xlabel('Rating')
            ax.set_ylabel('Density')
            ax.legend()
            ax.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('01_rating_distributions_kde.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 01_rating_distributions_kde.png")
        plt.close()
    
    def plot_correlation_heatmap(self):
        """Correlation heatmap for all key metrics"""
        # Select key metrics
        key_metrics = ['Rating', 'Gls', 'Ast', 'xg', 'xg_assist', 'sca', 'gca',
                      'progressive_passes', 'progressive_carries', 'passes_pct',
                      'touches']
        
        available = [m for m in key_metrics if m in self.df.columns]
        
        # Calculate correlation matrix
        corr_matrix = self.df[available].corr()
        
        # Create heatmap
        fig, ax = plt.subplots(figsize=(14, 12))
        
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                   center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
                   vmin=-1, vmax=1, ax=ax)
        
        ax.set_title('Correlation Heatmap: Key Metrics', fontsize=16, fontweight='bold', pad=20)
        
        plt.tight_layout()
        plt.savefig('02_correlation_heatmap.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 02_correlation_heatmap.png")
        plt.close()
    
    def plot_violin_by_result(self):
        """Violin plots by match result - more informative than box plots"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Violin Plots: Performance by Match Result', fontsize=16, fontweight='bold')
        axes = axes.flatten()
        
        metrics = ['Rating', 'Gls', 'Ast', 'xg', 'sca', 'progressive_passes']
        available = [m for m in metrics if m in self.df.columns]
        
        for idx, metric in enumerate(available):
            ax = axes[idx]
            
            # Prepare data
            plot_data = []
            for result_type in ['W', 'D', 'L']:
                mask = self.df['result'].str.contains(result_type, na=False) & self.df[metric].notna()
                if mask.sum() > 0:
                    temp_df = pd.DataFrame({
                        'Value': self.df.loc[mask, metric],
                        'Result': result_type
                    })
                    plot_data.append(temp_df)
            
            if plot_data:
                combined = pd.concat(plot_data, ignore_index=True)
                
                # Violin plot
                parts = ax.violinplot([combined[combined['Result'] == r]['Value'].values 
                                      for r in ['W', 'D', 'L'] if r in combined['Result'].unique()],
                                     positions=range(len(combined['Result'].unique())),
                                     showmeans=True, showmedians=True)
                
                # Color the violins
                colors = ['lightgreen', 'lightyellow', 'lightcoral']
                for pc, color in zip(parts['bodies'], colors):
                    pc.set_facecolor(color)
                    pc.set_alpha(0.7)
                
                ax.set_xticks(range(len(combined['Result'].unique())))
                ax.set_xticklabels([r for r in ['W', 'D', 'L'] if r in combined['Result'].unique()])
                ax.set_title(f'{metric}', fontsize=11, fontweight='bold')
                ax.set_ylabel(metric, fontsize=10)
                ax.set_xlabel('Match Result', fontsize=10)
                ax.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('03_violin_plots_by_result.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 03_violin_plots_by_result.png")
        plt.close()
    
    def plot_scatter_attacking_per_player(self):
        """Scatter plots for ATTACKING metrics - ONE PLOT PER PLAYER"""
        available = [m for m in self.attacking_metrics if m in self.df.columns and self.df[m].notna().sum() > 50]
        players = self.df['Player'].unique()
        
        for player in players:
            player_data = self.df[self.df['Player'] == player]
            
            fig, axes = plt.subplots(2, 4, figsize=(24, 12))
            fig.suptitle(f'{player} - ATTACKING Metrics vs Rating', fontsize=18, fontweight='bold')
            axes = axes.flatten()
            
            for idx, metric in enumerate(available[:8]):
                ax = axes[idx]
                mask = player_data[metric].notna() & player_data['Rating'].notna()
                
                if mask.sum() > 5:
                    x = player_data.loc[mask, metric]
                    y = player_data.loc[mask, 'Rating']
                    
                    ax.scatter(x, y, alpha=0.6, s=60, color='red', edgecolors='black', linewidth=1)
                    
                    # Trend line
                    if len(x) > 2:
                        z = np.polyfit(x, y, 1)
                        p = np.poly1d(z)
                        ax.plot(x, p(x), "darkred", linestyle='--', alpha=0.8, linewidth=2)
                    
                    # Pearson and Spearman correlation
                    corr_p, _ = stats.pearsonr(x, y)
                    corr_s, _ = spearmanr(x, y)
                    
                    ax.set_xlabel(metric, fontsize=11, fontweight='bold')
                    ax.set_ylabel('Rating', fontsize=11, fontweight='bold')
                    ax.set_title(f'{metric}\nPearson r={corr_p:.3f}, Spearman ρ={corr_s:.3f}', fontsize=10)
                    ax.grid(alpha=0.3)
                else:
                    ax.text(0.5, 0.5, 'Insufficient data', 
                           ha='center', va='center', transform=ax.transAxes, fontsize=12)
                    ax.set_title(metric)
            
            # Remove empty subplots
            for idx in range(len(available), 8):
                fig.delaxes(axes[idx])
            
            plt.tight_layout()
            plt.savefig(f'04_{player}_attacking_scatter.png', dpi=300, bbox_inches='tight')
            print(f"✅ Saved: 04_{player}_attacking_scatter.png")
            plt.close()
    
    def plot_scatter_creative_per_player(self):
        """Scatter plots for CREATIVE metrics - ONE PLOT PER PLAYER"""
        available = [m for m in self.creative_metrics if m in self.df.columns and self.df[m].notna().sum() > 50]
        players = self.df['Player'].unique()
        
        for player in players:
            player_data = self.df[self.df['Player'] == player]
            
            fig, axes = plt.subplots(2, 4, figsize=(24, 12))
            fig.suptitle(f'{player} - CREATIVE Metrics vs Rating', fontsize=18, fontweight='bold')
            axes = axes.flatten()
            
            for idx, metric in enumerate(available[:8]):
                ax = axes[idx]
                mask = player_data[metric].notna() & player_data['Rating'].notna()
                
                if mask.sum() > 5:
                    x = player_data.loc[mask, metric]
                    y = player_data.loc[mask, 'Rating']
                    
                    ax.scatter(x, y, alpha=0.6, s=60, color='blue', edgecolors='black', linewidth=1)
                    
                    # Trend line
                    if len(x) > 2:
                        z = np.polyfit(x, y, 1)
                        p = np.poly1d(z)
                        ax.plot(x, p(x), "darkblue", linestyle='--', alpha=0.8, linewidth=2)
                    
                    # Pearson and Spearman correlation
                    corr_p, _ = stats.pearsonr(x, y)
                    corr_s, _ = spearmanr(x, y)
                    
                    ax.set_xlabel(metric, fontsize=11, fontweight='bold')
                    ax.set_ylabel('Rating', fontsize=11, fontweight='bold')
                    ax.set_title(f'{metric}\nPearson r={corr_p:.3f}, Spearman ρ={corr_s:.3f}', fontsize=10)
                    ax.grid(alpha=0.3)
                else:
                    ax.text(0.5, 0.5, 'Insufficient data', 
                           ha='center', va='center', transform=ax.transAxes, fontsize=12)
                    ax.set_title(metric)
            
            # Remove empty subplots
            for idx in range(len(available), 8):
                fig.delaxes(axes[idx])
            
            plt.tight_layout()
            plt.savefig(f'05_{player}_creative_scatter.png', dpi=300, bbox_inches='tight')
            print(f"✅ Saved: 05_{player}_creative_scatter.png")
            plt.close()
    
    def plot_discipline_analysis(self):
        """Analyze relationship between discipline (cards) and ratings"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Discipline Analysis: Cards vs Rating', fontsize=16, fontweight='bold')
        
        # 1. Yellow cards vs rating
        ax = axes[0, 0]
        if 'cards_yellow' in self.df.columns:
            mask = self.df['cards_yellow'].notna() & self.df['Rating'].notna()
            if mask.sum() > 10:
                with_card = self.df.loc[mask & (self.df['cards_yellow'] > 0), 'Rating']
                without_card = self.df.loc[mask & (self.df['cards_yellow'] == 0), 'Rating']
                
                if len(with_card) > 0 and len(without_card) > 0:
                    bp = ax.boxplot([without_card, with_card], labels=['No Card', 'Yellow Card'], patch_artist=True)
                    for patch in bp['boxes']:
                        patch.set_facecolor('lightyellow')
                    ax.set_ylabel('Rating')
                    ax.set_title('Rating: With vs Without Yellow Card')
                    ax.grid(alpha=0.3)
                    
                    # T-test
                    t_stat, p_val = stats.ttest_ind(with_card, without_card)
                    ax.text(0.5, 0.95, f't-test: t={t_stat:.3f}, p={p_val:.4f}', 
                           transform=ax.transAxes, ha='center', va='top', fontweight='bold')
        
        # 2. Cards by player
        ax = axes[0, 1]
        players = self.df['Player'].unique()
        if 'cards_yellow' in self.df.columns:
            yellow_counts = [self.df[self.df['Player'] == p]['cards_yellow'].sum() for p in players]
            ax.bar(players, yellow_counts, color='yellow', edgecolor='black', alpha=0.7)
            ax.set_ylabel('Total Yellow Cards')
            ax.set_title('Total Yellow Cards by Player')
            ax.grid(alpha=0.3)
        
        # 3. Average rating when booked
        ax = axes[1, 0]
        if 'cards_yellow' in self.df.columns:
            ratings_when_booked = []
            labels = []
            for player in players:
                player_data = self.df[self.df['Player'] == player]
                mask = (player_data['cards_yellow'] > 0) & player_data['Rating'].notna()
                if mask.sum() > 0:
                    avg = player_data.loc[mask, 'Rating'].mean()
                    ratings_when_booked.append(avg)
                    labels.append(player)
            
            if ratings_when_booked:
                ax.bar(labels, ratings_when_booked, color='orange', edgecolor='black', alpha=0.7)
                ax.set_ylabel('Average Rating')
                ax.set_title('Average Rating When Receiving Yellow Card')
                ax.axhline(self.df['Rating'].mean(), color='red', linestyle='--', linewidth=2, label='Overall Avg')
                ax.legend()
                ax.grid(alpha=0.3)
        
        # 4. Correlation scatter
        ax = axes[1, 1]
        if 'cards_yellow' in self.df.columns:
            mask = self.df['cards_yellow'].notna() & self.df['Rating'].notna()
            if mask.sum() > 10:
                x = self.df.loc[mask, 'cards_yellow']
                y = self.df.loc[mask, 'Rating']
                
                ax.scatter(x, y, alpha=0.5, s=50, color='orange', edgecolors='black')
                
                corr_p, _ = stats.pearsonr(x, y)
                corr_s, _ = spearmanr(x, y)
                
                ax.set_xlabel('Yellow Cards')
                ax.set_ylabel('Rating')
                ax.set_title(f'Cards vs Rating\nPearson r={corr_p:.3f}, Spearman ρ={corr_s:.3f}')
                ax.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('06_discipline_analysis.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 06_discipline_analysis.png")
        plt.close()
    
    def plot_player_comparison(self):
        """Compare players across key metrics"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Player Comparison - Key Metrics', fontsize=16, fontweight='bold')
        
        players = self.df['Player'].unique()
        
        # 1. Average Rating
        ax = axes[0, 0]
        ratings = [self.df[self.df['Player'] == p]['Rating'].mean() for p in players]
        ax.bar(players, ratings, color='skyblue', alpha=0.8, edgecolor='black')
        ax.set_ylabel('Average Rating')
        ax.set_title('Average Rating by Player')
        ax.grid(alpha=0.3)
        
        # 2. Total Goals
        ax = axes[0, 1]
        goals = [self.df[self.df['Player'] == p]['Gls'].sum() for p in players]
        ax.bar(players, goals, color='green', alpha=0.8, edgecolor='black')
        ax.set_ylabel('Total Goals')
        ax.set_title('Total Goals by Player')
        ax.grid(alpha=0.3)
        
        # 3. Total Assists
        ax = axes[1, 0]
        assists = [self.df[self.df['Player'] == p]['Ast'].sum() for p in players]
        ax.bar(players, assists, color='orange', alpha=0.8, edgecolor='black')
        ax.set_ylabel('Total Assists')
        ax.set_title('Total Assists by Player')
        ax.grid(alpha=0.3)
        
        # 4. Average SCA
        ax = axes[1, 1]
        if 'sca' in self.df.columns:
            sca_avg = [self.df[self.df['Player'] == p]['sca'].mean() for p in players]
            ax.bar(players, sca_avg, color='purple', alpha=0.8, edgecolor='black')
            ax.set_ylabel('Average SCA per Match')
            ax.set_title('Average Shot-Creating Actions by Player')
            ax.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('07_player_comparison.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 07_player_comparison.png")
        plt.close()
    
    def plot_individual_ratings_by_result(self):
        """Individual player ratings by match result"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Individual Player Ratings by Match Result', fontsize=16, fontweight='bold')
        
        players = self.df['Player'].unique()
        result_types = ['W', 'D', 'L']
        colors = {'W': 'green', 'D': 'orange', 'L': 'red'}
        
        for idx, player in enumerate(players):
            ax = axes[idx // 2, idx % 2]
            player_data = self.df[self.df['Player'] == player]
            
            ratings_by_result = []
            labels = []
            bar_colors = []
            
            for result in result_types:
                mask = player_data['result'].str.contains(result, na=False)
                if mask.sum() > 0:
                    avg_rating = player_data.loc[mask, 'Rating'].mean()
                    count = mask.sum()
                    ratings_by_result.append(avg_rating)
                    labels.append(f'{result}\n(n={count})')
                    bar_colors.append(colors[result])
            
            bars = ax.bar(labels, ratings_by_result, color=bar_colors, alpha=0.7, edgecolor='black', linewidth=1.5)
            
            for bar, rating in zip(bars, ratings_by_result):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{rating:.2f}',
                       ha='center', va='bottom', fontweight='bold', fontsize=10)
            
            ax.set_ylabel('Average Rating', fontsize=11)
            ax.set_title(f'{player}', fontsize=12, fontweight='bold')
            ax.set_ylim(6, 9)
            ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('08_individual_ratings_by_result.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 08_individual_ratings_by_result.png")
        plt.close()
    
    def plot_attacking_vs_creative_impact(self):
        """Compare attacking vs creative impact for each player"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Attacking vs Creative Impact on Rating', fontsize=16, fontweight='bold')
        
        players = self.df['Player'].unique()
        
        for idx, player in enumerate(players):
            ax = axes[idx // 2, idx % 2]
            player_data = self.df[self.df['Player'] == player]
            
            # Calculate average correlations
            attacking_corrs = []
            creative_corrs = []
            
            for metric in self.attacking_metrics:
                if metric in player_data.columns:
                    mask = player_data[metric].notna() & player_data['Rating'].notna()
                    if mask.sum() > 5:
                        corr, _ = stats.pearsonr(
                            player_data.loc[mask, metric],
                            player_data.loc[mask, 'Rating']
                        )
                        attacking_corrs.append(abs(corr))
            
            for metric in self.creative_metrics:
                if metric in player_data.columns:
                    mask = player_data[metric].notna() & player_data['Rating'].notna()
                    if mask.sum() > 5:
                        corr, _ = stats.pearsonr(
                            player_data.loc[mask, metric],
                            player_data.loc[mask, 'Rating']
                        )
                        creative_corrs.append(abs(corr))
            
            avg_attacking = np.mean(attacking_corrs) if attacking_corrs else 0
            avg_creative = np.mean(creative_corrs) if creative_corrs else 0
            
            # Bar chart
            categories = ['Attacking', 'Creative']
            values = [avg_attacking, avg_creative]
            colors_bars = ['red', 'blue']
            
            bars = ax.bar(categories, values, color=colors_bars, alpha=0.7, edgecolor='black', linewidth=2)
            
            # Add value labels
            for bar, val in zip(bars, values):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{val:.3f}',
                       ha='center', va='bottom', fontweight='bold', fontsize=11)
            
            ax.set_ylabel('Average Absolute Correlation with Rating', fontsize=10)
            ax.set_title(f'{player}', fontsize=12, fontweight='bold')
            ax.set_ylim(0, max(values) * 1.2 if values else 1)
            ax.grid(axis='y', alpha=0.3)
            
            # Add interpretation
            if avg_attacking > avg_creative * 1.1:
                ax.text(0.5, 0.95, '→ ATTACKING dominant', 
                       transform=ax.transAxes, ha='center', fontweight='bold', color='red', fontsize=10)
            elif avg_creative > avg_attacking * 1.1:
                ax.text(0.5, 0.95, '→ CREATIVE dominant', 
                       transform=ax.transAxes, ha='center', fontweight='bold', color='blue', fontsize=10)
            else:
                ax.text(0.5, 0.95, '→ BALANCED', 
                       transform=ax.transAxes, ha='center', fontweight='bold', color='green', fontsize=10)
        
        plt.tight_layout()
        plt.savefig('09_attacking_vs_creative_impact.png', dpi=300, bbox_inches='tight')
        print("✅ Saved: 09_attacking_vs_creative_impact.png")
        plt.close()
    
    def generate_summary(self):
        """Generate text summary"""
        print("\n" + "="*60)
        print("SUMMARY REPORT")
        print("="*60)
        
        summary = f"""
FC BARCELONA PLAYER ANALYSIS - 2024/25 SEASON
ENHANCED EDA WITH ADVANCED TECHNIQUES
==============================================

DATASET:
- Total Records: {len(self.df)}
- Players Analyzed: {', '.join(self.df['Player'].unique())}
- Date Range: {self.df['Date'].min().strftime('%Y-%m-%d')} to {self.df['Date'].max().strftime('%Y-%m-%d')}

METHODS APPLIED:
✓ Kernel Density Estimation (KDE)
✓ Violin Plots
✓ Pearson & Spearman Correlation
✓ Correlation Heatmap
✓ Z-Score Standardization
✓ IQR Outlier Detection

VISUALIZATIONS CREATED:
01. Rating distributions with KDE
02. Correlation heatmap (all metrics)
03. Violin plots by match result
04-05. Attacking/Creative scatter plots per player (8 files)
06. Discipline analysis
07. Player comparison
08. Individual ratings by result
09. Attacking vs Creative impact

Total: 15 visualization files
"""
        
        print(summary)
        
        with open('EDA_SUMMARY_ENHANCED.txt', 'w', encoding='utf-8') as f:
            f.write(summary)
        
        print("\n✅ Summary saved to: EDA_SUMMARY_ENHANCED.txt")


def main():
    """Main EDA workflow"""
    print("\n" + "="*70)
    print(" FC BARCELONA PLAYER ANALYSIS")
    print(" ENHANCED EXPLORATORY DATA ANALYSIS")
    print("="*70)
    
    # Load data
    data_file = 'fbref_sofascore_merged_by_date_only.csv'
    
    eda = BarcelonaEDA(data_file)
    
    # Run analyses
    eda.data_overview()
    eda.player_summary()
    eda.match_result_analysis()
    eda.correlation_analysis()
    eda.outlier_detection()
    
    # Create visualizations
    eda.visualize_ratings()
    
    # Generate summary
    eda.generate_summary()
    
    print("\n" + "="*70)
    print(" ENHANCED EDA COMPLETE! 🎉")
    print("="*70)
    print("\nNew methods added:")
    print("  ✓ Kernel Density Estimation (KDE)")
    print("  ✓ Violin Plots")
    print("  ✓ Spearman Rank Correlation")
    print("  ✓ Correlation Heatmap")
    print("  ✓ Z-Score Standardization")
    print("  ✓ IQR Outlier Detection")


if __name__ == "__main__":
    main()


 FC BARCELONA PLAYER ANALYSIS
 ENHANCED EXPLORATORY DATA ANALYSIS
Loaded 223 matches for 4 players
Date range: 2024-08-17 00:00:00 to 2025-05-25 00:00:00

DATA OVERVIEW

Shape: (223, 39)

Columns: ['Player', 'Date', 'dayofweek', 'Competition', 'round', 'venue', 'result', 'Team', 'Opponent', 'Started', 'position', 'Min', 'Gls', 'Ast', 'pens_made', 'pens_att', 'shots', 'shots_on_target', 'cards_yellow', 'cards_red', 'touches', 'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xg_assist', 'sca', 'gca', 'passes_completed', 'passes', 'passes_pct', 'progressive_passes', 'carries', 'progressive_carries', 'take_ons', 'take_ons_won', 'match_report', 'Rating']

Missing Values:
position                2
touches                27
tackles                27
blocks                 27
xg                     27
npxg                   27
xg_assist              27
sca                    27
gca                    27
passes_completed       27
passes                 27
passes_pct             27
progress

In [28]:
"""
FC Barcelona Player Analysis - Hypothesis Testing (Refined Final)
Chain of Impact: Advanced Metrics (inc. Progressive Carries) → Player Rating → Team Result
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, shapiro, levene
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)


class BarcelonaHypothesisTesting:
    def __init__(self, data_file):
        """Load data"""
        self.df = pd.read_csv(data_file)
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        
        # --- REFINED METRIC LIST (UPDATED) ---
        # "Finisher" (Attacking)
        self.attacking_metrics = ['npxg', 'shots'] 
        
        # "Creator" (Creative) - Added 'progressive_carries'
        self.creative_metrics = [
            'xg_assist', 
            'sca', 
            'progressive_passes', 
            'take_ons_won',
            'progressive_carries'  # Included, but treated normally
        ]
        
        print(f"Loaded {len(self.df)} matches for {self.df['Player'].nunique()} players")
        print(f"Date range: {self.df['Date'].min()} to {self.df['Date'].max()}")
    
    def check_assumptions(self):
        """Check assumptions for parametric tests"""
        print("\n" + "="*70)
        print("ASSUMPTION CHECKS")
        print("="*70)
        
        # Normality
        data_rating = self.df['Rating'].dropna()
        stat, p = shapiro(data_rating)
        print(f"1. Rating Normality: W={stat:.4f}, p={p:.4f} ", end='')
        print("✓ Normal" if p > 0.05 else "✗ Non-normal (Using robust interpretation)")

        # Homogeneity
        w = self.df[self.df['result'].str.contains('W', na=False)]['Rating'].dropna()
        d = self.df[self.df['result'].str.contains('D', na=False)]['Rating'].dropna()
        l = self.df[self.df['result'].str.contains('L', na=False)]['Rating'].dropna()
        
        if len(w) > 0 and len(d) > 0 and len(l) > 0:
            stat, p = levene(w, d, l)
            print(f"2. Rating Variance (W/D/L): W={stat:.4f}, p={p:.4f} ", end='')
            print("✓ Equal" if p > 0.05 else "⚠ Unequal")
    
    def phase1_metrics_to_rating(self):
        """Phase 1: Metric Correlations (Including Progressive Carries)"""
        print("\n" + "="*70)
        print("PHASE 1: METRICS → RATING")
        print("="*70)
        
        results = []
        all_metrics = self.attacking_metrics + self.creative_metrics
        
        for player in self.df['Player'].unique():
            player_data = self.df[self.df['Player'] == player]
            print(f"\n{player}:")
            
            for metric in all_metrics:
                if metric in player_data.columns:
                    mask = player_data[metric].notna() & player_data['Rating'].notna()
                    if mask.sum() > 5:
                        corr, p = stats.pearsonr(player_data.loc[mask, metric], player_data.loc[mask, 'Rating'])
                        sig = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
                        print(f"  {metric:20s}: r = {corr:.3f} {sig}")
                        
                        results.append({
                            'Player': player,
                            'Metric': metric,
                            'Type': 'Attacking' if metric in self.attacking_metrics else 'Creative',
                            'Correlation': corr,
                            'Significant': p < 0.05
                        })
        
        self.plot_phase1_results(results)
        return pd.DataFrame(results)
    
    def plot_phase1_results(self, results):
        """Visualize correlations (Standard plotting for all metrics)"""
        if not results: return
        df_results = pd.DataFrame(results)
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Phase 1: What drives Rating? (Attacking vs Creative)', fontsize=16, fontweight='bold')
        
        players = df_results['Player'].unique()
        for idx, player in enumerate(players):
            if idx >= 4: break
            ax = axes[idx // 2, idx % 2]
            p_data = df_results[df_results['Player'] == player].sort_values('Correlation')
            
            # Standard colors based on type
            colors = ['#ff9999' if t == 'Attacking' else '#66b3ff' for t in p_data['Type']]
            
            # Plot bars (No special highlighting for progressive_carries)
            bars = ax.barh(p_data['Metric'], p_data['Correlation'], color=colors, edgecolor='black', alpha=0.8)
            
            # Add significance markers
            for i, (bar, sig) in enumerate(zip(bars, p_data['Significant'])):
                if sig:
                    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2, 
                           '*', va='center', fontweight='bold', fontsize=14, color='black')

            ax.set_title(player, fontweight='bold')
            ax.set_xlim(-0.1, 1.0)
            ax.grid(alpha=0.3, axis='x')
            
            # Legend on first plot
            if idx == 0:
                from matplotlib.patches import Patch
                legend_elements = [Patch(facecolor='#ff9999', label='Attacking'),
                                 Patch(facecolor='#66b3ff', label='Creative')]
                ax.legend(handles=legend_elements, loc='lower right')

        plt.tight_layout()
        plt.savefig('H1_metrics_correlation_final.png', dpi=300)
        print("\n✅ Saved: H1_metrics_correlation_final.png")
        plt.close()
    
    def phase2_rating_to_outcome(self):
        """Phase 2: Outcome Prediction (Including Total Progressive Carries)"""
        print("\n" + "="*70)
        print("PHASE 2: AGGREGATE IMPACT → RESULT")
        print("="*70)
        
        match_agg = self.create_match_aggregates()
        
        # Metrics to test (Added Total_PrgCarries)
        metrics = ['Total_npxG', 'Total_SCA', 'Total_Goals', 'Avg_Rating', 'Total_PrgCarries']
        
        anova_results = []
        for metric in metrics:
            if metric in match_agg.columns:
                groups = [match_agg[match_agg['result'].str.contains(r, na=False)][metric].dropna() for r in ['W', 'D', 'L']]
                if all(len(g) > 0 for g in groups):
                    f, p = f_oneway(*groups)
                    print(f"\n{metric}: ANOVA F={f:.3f}, p={p:.4f} {'*' if p<0.05 else ''}")
                    anova_results.append({'Metric': metric, 'Significant': p < 0.05})
        
        self.plot_phase2_results(match_agg, anova_results)

    def create_match_aggregates(self):
        agg_dict = {
            'npxg': 'sum', 
            'sca': 'sum', 
            'Gls': 'sum', 
            'Rating': 'mean', 
            'progressive_carries': 'sum', 
            'result': 'first'
        }
        agg_dict = {k: v for k, v in agg_dict.items() if k in self.df.columns}
        
        match_agg = self.df.groupby(['Date', 'Opponent']).agg(agg_dict).reset_index()
        
        rename_map = {
            'npxg': 'Total_npxG', 'sca': 'Total_SCA', 'Gls': 'Total_Goals', 
            'Rating': 'Avg_Rating', 'progressive_carries': 'Total_PrgCarries'
        }
        return match_agg.rename(columns=rename_map)

    def plot_phase2_results(self, match_agg, anova_results):
        # Increased to 5 subplots to include Carries (or dynamic based on list)
        metrics = ['Total_npxG', 'Total_SCA', 'Total_PrgCarries', 'Avg_Rating']
        fig, axes = plt.subplots(1, len(metrics), figsize=(20, 5)) 
        
        for idx, metric in enumerate(metrics):
            if metric in match_agg.columns:
                ax = axes[idx]
                
                # Prepare data for matplotlib boxplot
                data_to_plot = []
                labels = ['W', 'D', 'L']
                colors = ['#90EE90', '#FFFFE0', '#FFB6C1']
                
                for r in labels:
                    subset = match_agg[match_agg['result'].str.contains(r, na=False)][metric].dropna()
                    data_to_plot.append(subset.values)
                
                # Matplotlib boxplot
                bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True)
                
                for patch, color in zip(bp['boxes'], colors):
                    patch.set_facecolor(color)
                
                ax.set_title(metric)
                ax.grid(alpha=0.3)
                
                # Check sig
                is_sig = next((r['Significant'] for r in anova_results if r['Metric'] == metric), False)
                if is_sig:
                    ax.text(0.5, 0.9, 'Sig *', transform=ax.transAxes, ha='center', 
                           color='red', fontweight='bold', bbox=dict(facecolor='white', edgecolor='red', boxstyle='round'))

        plt.tight_layout()
        plt.savefig('H2_team_impact_final.png', dpi=300)
        print("\n✅ Saved: H2_team_impact_final.png")
        plt.close()

def main():
    # Replace with your actual filename
    data_file = 'fbref_sofascore_merged_by_date_only.csv'
    try:
        tester = BarcelonaHypothesisTesting(data_file)
        tester.check_assumptions()
        tester.phase1_metrics_to_rating()
        tester.phase2_rating_to_outcome()
        print("\nANALYSIS COMPLETE.")
    except FileNotFoundError:
        print("File not found.")

if __name__ == "__main__":
    main()

Loaded 223 matches for 4 players
Date range: 2024-08-17 00:00:00 to 2025-05-25 00:00:00

ASSUMPTION CHECKS
1. Rating Normality: W=0.9759, p=0.0007 ✗ Non-normal (Using robust interpretation)
2. Rating Variance (W/D/L): W=0.0389, p=0.9618 ✓ Equal

PHASE 1: METRICS → RATING

Lewandowski:
  npxg                : r = 0.543 ***
  shots               : r = 0.422 **
  xg_assist           : r = 0.329 *
  sca                 : r = 0.263 
  progressive_passes  : r = 0.060 
  take_ons_won        : r = 0.243 
  progressive_carries : r = 0.309 *

Pedri:
  npxg                : r = 0.341 *
  shots               : r = 0.328 *
  xg_assist           : r = 0.718 ***
  sca                 : r = 0.651 ***
  progressive_passes  : r = 0.760 ***
  take_ons_won        : r = 0.518 ***
  progressive_carries : r = 0.256 

Raphinha:
  npxg                : r = 0.418 **
  shots               : r = 0.354 **
  xg_assist           : r = 0.515 ***
  sca                 : r = 0.575 ***
  progressive_passes  : r = 0.414 